In [2]:
import numpy as np
import re
import pandas as pd
from datetime import datetime as dt, timedelta

In [3]:
#Read in collected/wrangled data 
geo_codes = pd.read_csv('_Addresses_with_Geocodes.csv')
df = pd.read_csv('_ARLCimeFinal.csv', encoding='latin-1')

In [4]:
#Merge data and apply crime category labels 
df2 = df.merge(geo_codes[['Address', 'Latitude', 'Longitude']], how='left', left_on='Address_FINAL', right_on='Address')

df2 = df2[df2['CategoryCode']!= 0]

df2['Crime_Category'] = np.select([(df2['CategoryCode'] == 1),
                            (df2['CategoryCode'] == 2),
                            (df2['CategoryCode'] == 3)],
                            
                            ['Crimes Against Person', 
                             'Crimes Against Property', 
                             'Crimes Against Society'], 
                             
                             default=np.nan)

In [5]:
#Drop NAs from merge
df3 = df2.dropna()

#Filter out odd addresses/incident dates 
df4 = df3.copy()
df4['IncidentDate'] = df4['IncidentDate'].astype('str')
df4 = df4[~df4['IncidentDate'].str.contains(r'[A-Za-z]')]

#Convert initial dtypes
df5 = df4.copy()

df5['date_reported'] = pd.to_datetime(df5['date_reported'])
df5['date_reported'] = df5['date_reported'].dt.strftime('%Y-%m%d')

df5["IncidentMonth"] = df5["IncidentMonth"].astype('int')

In [6]:
#Clean for incident date (account for incorrect input data from Arlington County)
df5['idate2'] = df5['IncidentDate'].astype('str')
df5['idate2'] = np.where(df5['IncidentDate'].str.isnumeric() == True, '0', df5['IncidentDate'])

df5['rough_day'] = df5['idate2'].str[-2:].astype('int')
df5['idate3'] = np.where((df5['rough_day'] > 31) | (df5['rough_day'] <= 0), '0', df5['idate2'])
df5['idate4'] = np.where((df5['IncidentMonth'].isin([4, 6, 9, 11])) & (df5['rough_day'] > 30), '0', df5['idate3'])


df5['idate5'] = np.where((df5["IncidentMonth"] == 2) & (df5["rough_day"] > 29), '0', df5['idate4'])

df5['idate6'] = np.where((df5['IncidentMonth'] > 12) | (df5['IncidentMonth'] < 1), '0', df5['idate5'])
df5['idate7'] = np.where (df5['idate6'].str.contains(r'^\d{3}-', regex=True), '0', df5['idate6'])

df5['idate8'] = np.where ((~df5['idate7'].str.contains(r'^\d{4}-\d{2}\d{2}$', regex=True)), '0', df5['idate7'])

df5['test_date'] = np.where(df5['idate8'] == '0', df5['date_reported'], df5['idate8'])

In [7]:
#Create final copy with incident date in tact and make final dtype adjustments
final_df = df5.copy()

final_df = final_df[['id', 'date_reported', 'details', 'CRIME_FINAL', 'Incident_FINAL',
       'Address_FINAL', 'ZIP_Code', 'CategoryCode', 'Address', 'Latitude', 'Longitude',
       'Crime_Category', 'test_date']].rename(columns={'test_date':'incident_date'})

final_df['incident_date'] = pd.to_datetime(final_df['incident_date'], format='%Y-%m%d')
final_df['date_reported'] = pd.to_datetime(final_df['date_reported'], format='%Y-%m%d')

#Cut off incorrect dates
final_df = final_df[final_df['incident_date']>='2016-01-01']

In [11]:
final_df['date_reported'] = pd.to_datetime(final_df['date_reported'])
final_df['incident_date'] = pd.to_datetime(final_df['incident_date'])
final_df['ZIP_Code'] = final_df['ZIP_Code'].astype('int').astype('str')

final_df['monthname'] = final_df['incident_date'].dt.month_name()
final_df['month_num'] = final_df['incident_date'].dt.month
final_df['year'] =final_df['incident_date'].dt.year
final_df['yearmonth'] = final_df['monthname'] + '-' + final_df['year'].astype('str')

final_df['dow_num'] = final_df['incident_date'].dt.dayofweek
final_df['dow'] = np.select([(final_df['dow_num'] == 0),
                             (final_df['dow_num'] == 1),
                             (final_df['dow_num'] == 2),
                             (final_df['dow_num'] == 3),
                             (final_df['dow_num'] == 4),
                             (final_df['dow_num'] == 5),
                             (final_df['dow_num'] == 6)], 

                             ['Monday', 'Tuesday', 'Wednesday', 
                              'Thursday', 'Friday', 'Saturday', 'Sunday'], 

                              default=np.nan)

In [12]:
final_df.dtypes

id                         int64
date_reported     datetime64[ns]
details                   object
CRIME_FINAL               object
Incident_FINAL            object
Address_FINAL             object
ZIP_Code                  object
CategoryCode               int64
Address                   object
Latitude                 float64
Longitude                float64
Crime_Category            object
incident_date     datetime64[ns]
monthname                 object
month_num                  int64
year                       int64
yearmonth                 object
dow_num                    int64
dow                       object
dtype: object

In [13]:
final_df.to_csv('final_data.csv', index=False)